<a href="https://colab.research.google.com/github/Parado-xy/eye-disease-classification/blob/master/eye_disease_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Herein we train a classifier for eye-disease detection.

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gunavenkatdoddi/eye-diseases-classification")

print("Path to dataset files:", path)

100%|██████████| 736M/736M [00:35<00:00, 21.9MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/gunavenkatdoddi/eye-diseases-classification/versions/1


In [3]:
# List the contents of the downloaded dataset directory
import os

kaggle_dataset_path = "/root/.cache/kagglehub/datasets/gunavenkatdoddi/eye-diseases-classification/versions/1"
print(os.listdir(kaggle_dataset_path))

# # Recursively list contents of subdirectories to find image files
# for root, dirs, files in os.walk(kaggle_dataset_path):
#     if files:
#         print(f"\nFiles in {root}:")
#         for f in files:
#             print(f)

['dataset']


Dataset Description:

The dataset consists of Normal, Diabetic Retinopathy, Cataract and Glaucoma retinal images where each class have approximately 1000 images. These images are collected from various sorces like IDRiD, Oculur recognition, HRF etc.

# Task
Write a CNN classifier using the dataset in the directory "Multi-Class-Eye-Disease-Dataset", with a resnet base, tensorflow framework, model checkpointing and early stopping.

## Prepare the dataset

### Subtask:
Load the images and labels from the downloaded dataset directory, organize the data into training, validation, and test sets, and create data pipelines for efficient loading during training.


**Reasoning**:
Define the path to the dataset, create a list of image paths and labels, and split the data into training, validation, and test sets.



In [4]:
# List the contents of the dataset directory to understand its structure
print(os.listdir(dataset_path))

# Also check contents of any subdirectories if they exist
for item in os.listdir(dataset_path):
    item_path = os.path.join(dataset_path, item)
    if os.path.isdir(item_path):
        print(f"\nContents of {item}:")
        print(os.listdir(item_path))

['dataset']

Contents of dataset:
['cataract', 'diabetic_retinopathy', 'glaucoma', 'normal']


In [12]:
import os
import glob
import pandas as pd
from sklearn.model_selection import train_test_split

# Define the corrected path to the dataset directory
dataset_path = "/root/.cache/kagglehub/datasets/gunavenkatdoddi/eye-diseases-classification/versions/1/dataset"

# Create lists for image file paths and labels
image_paths = []
labels = []

# Iterate through subdirectories to get image paths and labels
for class_dir in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_dir)
    if os.path.isdir(class_path):
        # Include both .jpg and .png files
        for image_file in glob.glob(os.path.join(class_path, "*.jpg")) + glob.glob(os.path.join(class_path, "*.png")):
            image_paths.append(image_file)
            labels.append(class_dir)

# Create a pandas DataFrame for easier manipulation
df = pd.DataFrame({'image_path': image_paths, 'label': labels})

# Split the data into training, validation, and test sets
train_df, temp_df = train_test_split(df, test_size=0.3, stratify=df['label'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)

print("Training set size:", len(train_df))
print("Validation set size:", len(val_df))
print("Test set size:", len(test_df))

Training set size: 2183
Validation set size: 468
Test set size: 468


In [13]:
import tensorflow as tf

# Define image dimensions
IMG_WIDTH = 128
IMG_HEIGHT = 128
AUTOTUNE = tf.data.AUTOTUNE

# Function to load and preprocess images
def preprocess_image(image_path, label):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])
    img = img / 255.0  # Normalize to [0, 1]
    return img, label

# Function for data augmentation (apply only to training data)
def augment_image(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, lower=0.9, upper=1.1)
    return image, label

# Create tf.data.Dataset for each set
train_ds = tf.data.Dataset.from_tensor_slices((train_df['image_path'].values, train_df['label'].values))
val_ds = tf.data.Dataset.from_tensor_slices((val_df['image_path'].values, val_df['label'].values))
test_ds = tf.data.Dataset.from_tensor_slices((test_df['image_path'].values, test_df['label'].values))

# Map preprocessing and augmentation
train_ds = train_ds.map(preprocess_image, num_parallel_calls=AUTOTUNE)
train_ds = train_ds.map(augment_image, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(preprocess_image, num_parallel_calls=AUTOTUNE)
test_ds = test_ds.map(preprocess_image, num_parallel_calls=AUTOTUNE)

# One-hot encode labels
def one_hot_label(image, label):
    class_names = sorted(df['label'].unique())
    one_hot = tf.one_hot(tf.argmax(tf.constant(class_names) == label), len(class_names))
    return image, one_hot

train_ds = train_ds.map(one_hot_label, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(one_hot_label, num_parallel_calls=AUTOTUNE)
test_ds = test_ds.map(one_hot_label, num_parallel_calls=AUTOTUNE)


# Batch and prefetch the datasets
BATCH_SIZE = 32
train_ds = train_ds.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

print("Data pipelines created successfully.")

Data pipelines created successfully.


## Build the cnn model

### Subtask:
Define a CNN model using TensorFlow and a pre-trained ResNet base. Add layers for classification on top of the ResNet base.


In [14]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.models import Model

# Define the number of classes
num_classes = len(df['label'].unique())

# Load a pre-trained ResNet base model
base_model = ResNet50V2(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
                        include_top=False,
                        weights='imagenet')

# Freeze the layers of the pre-trained ResNet base
base_model.trainable = False

# Create a Sequential model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Print a summary of the model architecture
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50v2 (Functional)         │ (None, 4, 4, 2048)     │    23,564,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,827,459 (90.89 MB)

 Trainable params: 262,659 (1.00 MB)

 Non-trainable params: 23,564,800 (89.89 MB)

## Configure the training process

### Subtask:
Set up the optimizer, loss function, and metrics for training. Implement model checkpointing to save the best model during training and early stopping to prevent overfitting.


In [15]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Compile the model
model.compile(optimizer=Adam(),
              loss=CategoricalCrossentropy(),
              metrics=[CategoricalAccuracy()])

# Define ModelCheckpoint callback
checkpoint_path = "best_model.weights.h5" # Define the path to save the best model weights
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor='val_categorical_accuracy', # Monitor validation accuracy
    mode='max', # Save when validation accuracy is maximized
    save_best_only=True)

# Define EarlyStopping callback
early_stopping_callback = EarlyStopping(
    monitor='val_categorical_accuracy', # Monitor validation accuracy
    patience=10, # Stop after 10 epochs with no improvement
    mode='max', # Based on maximizing validation accuracy
    restore_best_weights=True) # Restore model weights from the best epoch

print("Model compiled and callbacks defined.")

Model compiled and callbacks defined.


## Train the model

### Subtask:
Train the CNN model using the prepared dataset and configurations. Monitor the training progress using the specified metrics.


In [16]:
EPOCHS = 50
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[model_checkpoint_callback, early_stopping_callback]
)

Epoch 1/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 35s 380ms/step - categorical_accuracy: 0.6585 - loss: 0.8632 - val_categorical_accuracy: 0.7821 - val_loss: 0.5057
Epoch 2/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 15s 225ms/step - categorical_accuracy: 0.7663 - loss: 0.5298 - val_categorical_accuracy: 0.8034 - val_loss: 0.4605
Epoch 3/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 15s 221ms/step - categorical_accuracy: 0.8084 - loss: 0.4386 - val_categorical_accuracy: 0.8120 - val_loss: 0.4305
Epoch 4/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 15s 220ms/step - categorical_accuracy: 0.8393 - loss: 0.3858 - val_categorical_accuracy: 0.8355 - val_loss: 0.4082
Epoch 5/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 20s 211ms/step - categorical_accuracy: 0.8423 - loss: 0.3699 - val_categorical_accuracy: 0.8355 - val_loss: 0.4263
Epoch 6/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 21s 221ms/step - categorical_accuracy: 0.8538 - loss: 0.3322 - val_categorical_accuracy: 0.8568 - val_loss: 0.3862
Epoch 7/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 15s 222ms/step - categorical_accuracy: 0.871

## Evaluate the model

### Subtask:
Evaluate the trained model on the test set to assess its performance.


In [17]:
loss, accuracy = model.evaluate(test_ds)

print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 178ms/step - categorical_accuracy: 0.8432 - loss: 0.3614
Test Loss: 0.39280709624290466
Test Accuracy: 0.8269230723381042


In [18]:
# Download the best model weights
from google.colab import files

files.download('/content/best_model.weights.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Summary:

### Data Analysis Key Findings

* The dataset was successfully loaded, split into training (70%), validation (15%), and test (15%) sets, and processed into efficient TensorFlow data pipelines with image resizing, normalization, augmentation (for training), and one-hot encoding.
* A CNN model was built using a pre-trained ResNet50V2 base with its weights frozen, and a classification head consisting of global average pooling and two dense layers was added.
* The model was compiled using the Adam optimizer, Categorical Crossentropy loss, and Categorical Accuracy metric.
* Model checkpointing was implemented to save the best model weights based on validation accuracy.
* Early stopping was configured to halt training if validation accuracy did not improve for 10 epochs, restoring the best weights.
* The model was trained for 17 epochs (due to early stopping), achieving a training accuracy of 0.9465 and a validation accuracy of 0.8098 at the end of training.
* The final evaluation on the test set resulted in a test accuracy of approximately 0.8369 and a test loss of approximately 0.3828.

### Insights or Next Steps

* Investigate the discrepancy between training accuracy (0.9465) and validation and test accuracy (around 0.81-0.84), which might indicate some overfitting despite early stopping. Techniques like fine-tuning the ResNet base or adding more regularization could be explored.
* Perform a more detailed evaluation by calculating precision, recall, and F1-score for each class to understand the model's performance on different eye disease categories.
* Visualize the training and validation accuracy and loss curves to get a better understanding of the training process and identify potential issues.